# How to return citations

How can we get a model to cite which parts of the source documents it referenced in its response?

To explore some techniques for extracting citations, let's first create a simple RAG chain. To start we'll just retrieve from the web using the [TavilySearchAPIRetriever](https://js.langchain.com/docs/integrations/retrievers/tavily).

## Setup
### Dependencies

We’ll use an OpenAI chat model and embeddings and a Memory vector store in this walkthrough, but everything shown here works with any [ChatModel](/docs/modules/model_io/chat) or [LLM](/docs/modules/model_io/llms), [Embeddings](https://js.langchain.com/docs/modules/data_connection/text_embedding/), and [VectorStore](https://js.langchain.com/docs/modules/data_connection/vectorstores/) or [Retriever](/docs/modules/data_connection/retrievers/).

We’ll use the following packages:

```bash
npm install --save langchain @langchain/community @langchain/openai
```

We need to set environment variables for Tavily Search & OpenAI:

```bash
export OPENAI_API_KEY=YOUR_KEY
export TAVILY_API_KEY=YOUR_KEY
```

### LangSmith

Many of the applications you build with LangChain will contain multiple steps with multiple invocations of LLM calls. As these applications get more and more complex, it becomes crucial to be able to inspect what exactly is going on inside your chain or agent. The best way to do this is with [LangSmith](https://smith.langchain.com/).

Note that LangSmith is not needed, but it is helpful. If you do want to use LangSmith, after you sign up at the link above, make sure to set your environment variables to start logging traces:


```bash
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=YOUR_KEY
```

### Initial setup

In [5]:
import { TavilySearchAPIRetriever } from "@langchain/community/retrievers/tavily_search_api";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-3.5-turbo",
  temperature: 0,
});
const retriever = new TavilySearchAPIRetriever({
  k: 6,
});
const prompt = ChatPromptTemplate.fromMessages([
  ["system", "You're a helpful AI assistant. Given a user question and some web article snippets, answer the user question. If none of the articles answer the question, just say you don't know.\n\nHere are the web articles:{context}"],
  ["human", "{question}"],
])

Now that we've got a model, retriever and prompt, let's chain them all together. We'll need to add some logic for formatting our retrieved `Document`s to a string that can be passed to our prompt. We'll make it so our chain returns both the answer and the retrieved Documents.

In [9]:
import { Document } from "@langchain/core/documents";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { RunnableMap, RunnablePassthrough } from "@langchain/core/runnables";

/**
 * Format the documents into a readable string.
 */
const formatDocs = (input: Record<string, any>): string => {
  const { docs } = input;
  return "\n\n" + docs.map((doc: Document) => `Article title: ${doc.metadata.title}\nArticle Snippet: ${doc.pageContent}`).join("\n\n");
}
// subchain for generating an answer once we've done retrieval
const answerChain = prompt.pipe(llm).pipe(new StringOutputParser());
const map = RunnableMap.from({
  question: new RunnablePassthrough(),
  docs: retriever,
})
// complete chain that calls the retriever -> formats docs to string -> runs answer subchain -> returns just the answer and retrieved docs.
const chain = map.assign({ context: formatDocs }).assign({ answer: answerChain }).pick(["answer", "docs"])

In [11]:
await chain.invoke("How fast are cheetahs?")

{
  answer: "Cheetahs are capable of reaching speeds as high as 75 mph or 120 km/h. Their average speed, however,"... 29 more characters,
  docs: [
    Document {
      pageContent: "Now, their only hope lies in the hands of human conservationists, working tirelessly to save the che"... 880 more characters,
      metadata: {
        title: "How Fast Are Cheetahs, and Other Fascinating Facts About the World's ...",
        source: "https://www.discovermagazine.com/planet-earth/how-fast-are-cheetahs-and-other-fascinating-facts-abou"... 21 more characters,
        score: 0.93715,
        images: null
      }
    },
    Document {
      pageContent: "If a lion comes along, the cheetah will abandon its catch -- it can't fight off a lion, and chances "... 911 more characters,
      metadata: {
        title: "What makes a cheetah run so fast? | HowStuffWorks",
        source: "https://animals.howstuffworks.com/mammals/cheetah-speed.htm",
        score: 0.93412,
        images: null
      }
 

LangSmith trace [here](https://smith.langchain.com/public/bb0ed37e-b2be-4ae9-8b0d-ce2aff0b4b5e/r)

## Function-calling

### Cite documents
Let's try using [OpenAI function-calling](/docs/modules/model_io/chat/function_calling) to make the model specify which of the provided documents it's actually referencing when answering. LangChain has some utils for converting objects or zod objects to the JSONSchema format expected by OpenAI, so we'll use that to define our functions:

In [12]:
import { z } from "zod";
import { StructuredTool } from "@langchain/core/tools";
import { formatToOpenAITool } from "@langchain/openai";

class CitedAnswer extends StructuredTool {
  name = "cited_answer";
  
  description = "Answer the user question based only on the given sources, and cite the sources used.";

  schema = z.object({
    answer: z.string().describe("The answer to the user question, which is based only on the given sources."),
    citations: z.array(z.number()).describe("The integer IDs of the SPECIFIC sources which justify the answer.")
  });

  constructor() {
    super();
  }

  _call(input: z.infer<typeof this["schema"]>): Promise<string> {
    return Promise.resolve(JSON.stringify(input, null, 2));
  }
}

const asOpenAITool = formatToOpenAITool(new CitedAnswer());
const tools1 = [asOpenAITool];

Let's see what the model output is like when we pass in our functions and a user input:

In [13]:
const llmWithTool1 = llm.bind({
  tools: tools1,
  tool_choice: asOpenAITool
});

const exampleQ = `What Brian's height?

Source: 1
Information: Suzy is 6'2"

Source: 2
Information: Jeremiah is blonde

Source: 3
Information: Brian is 3 inches shorted than Suzy`;

await llmWithTool1.invoke(exampleQ);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "",
    additional_kwargs: {
      function_call: undefined,
      tool_calls: [
        {
          id: "call_WzPoDCIRQ1pCah8k93cVrqex",
          type: "function",
          function: [Object]
        }
      ]
    }
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "",
  name: undefined,
  additional_kwargs: {
    function_call: undefined,
    tool_calls: [
      {
        id: "call_WzPoDCIRQ1pCah8k93cVrqex",
        type: "function",
        function: {
          name: "cited_answer",
          arguments: "{\n" +
            `  "answer": "Brian's height is 6'2\\" - 3 inches",\n` +
            '  "citations": [1, 3]\n' +
            "}"
        }
      }
    ]
  }
}

LangSmith trace [here](https://smith.langchain.com/public/34441213-cbb9-4775-a67e-2294aa1ccf69/r)

We'll add an output parser to convert the OpenAI API response to a nice object. We use the [JsonOutputKeyToolsParser](https://api.js.langchain.com/classes/langchain_output_parsers.JsonOutputKeyToolsParser.html) for this:

In [14]:
import { JsonOutputKeyToolsParser } from "langchain/output_parsers";

const outputParser = new JsonOutputKeyToolsParser({ keyName: "cited_answer", returnSingle: true });

await llmWithTool1.pipe(outputParser).invoke(exampleQ);

{ answer: `Brian's height is 6'2" - 3 inches`, citations: [ 1, 3 ] }

LangSmith trace [here](https://smith.langchain.com/public/1a045c25-ec5c-49f5-9756-6022edfea6af/r)

Now we're ready to put together our chain

In [16]:
import { Document } from "@langchain/core/documents";

const formatDocsWithId = (docs: Array<Document>): string => {
  return "\n\n" + docs.map((doc: Document, idx: number) => `Source ID: ${idx}\nArticle title: ${doc.metadata.title}\nArticle Snippet: ${doc.pageContent}`).join("\n\n");
}
// subchain for generating an answer once we've done retrieval
const answerChain1 = prompt.pipe(llmWithTool1).pipe(outputParser);
const map1 = RunnableMap.from({
  question: new RunnablePassthrough(),
  docs: retriever,
})
// complete chain that calls the retriever -> formats docs to string -> runs answer subchain -> returns just the answer and retrieved docs.
const chain1 = map1
  .assign({ context: (input: { docs: Array<Document> }) => formatDocsWithId(input.docs) })
  .assign({ cited_answer: answerChain1 })
  .pick(["cited_answer", "docs"])

In [17]:
await chain1.invoke("How fast are cheetahs?")

{
  cited_answer: {
    answer: "Cheetahs can reach speeds of up to 75 mph (120 km/h).",
    citations: [ 3 ]
  },
  docs: [
    Document {
      pageContent: "The speeds attained by the cheetah may be only slightly greater than those achieved by the pronghorn"... 2527 more characters,
      metadata: {
        title: "Cheetah - Wikipedia",
        source: "https://en.wikipedia.org/wiki/Cheetah",
        score: 0.97773,
        images: null
      }
    },
    Document {
      pageContent: "Contact Us − +\n" +
        "Address\n" +
        "Smithsonian's National Zoo & Conservation Biology Institute  3001 Connecticut"... 1343 more characters,
      metadata: {
        title: "Cheetah | Smithsonian's National Zoo and Conservation Biology Institute",
        source: "https://nationalzoo.si.edu/animals/cheetah",
        score: 0.9681,
        images: null
      }
    },
    Document {
      pageContent: "The maximum speed cheetahs have been measured at is 114 km (71 miles) per hour, and th

LangSmith trace [here](https://smith.langchain.com/public/2a29cfd6-89fa-45bb-9b2a-f730e81061c2/r)

### Cite snippets

What if we want to cite actual text spans? We can try to get our model to return these, too.

*Aside: Note that if we break up our documents so that we have many documents with only a sentence or two instead of a few long documents, citing documents becomes roughly equivalent to citing snippets, and may be easier for the model because the model just needs to return an identifier for each snippet instead of the actual text. Probably worth trying both approaches and evaluating.*

In [3]:
const citationSchema = z.object({
  sourceId: z.number().describe("The integer ID of a SPECIFIC source which justifies the answer."),
  quote: z.string().describe("The VERBATIM quote from the specified source that justifies the answer.")
})

class QuotedAnswer extends StructuredTool {
  name = "quoted_answer";
  
  description = "Answer the user question based only on the given sources, and cite the sources used.";

  schema = z.object({
    answer: z.string().describe("The answer to the user question, which is based only on the given sources."),
    citations: z.array(citationSchema).describe("Citations from the given sources that justify the answer.")
  });

  constructor() {
    super();
  }

  _call(input: z.infer<typeof this["schema"]>): Promise<string> {
    return Promise.resolve(JSON.stringify(input, null, 2));
  }
}

const quotedAnswerTool = formatToOpenAITool(new QuotedAnswer());
const tools2 = [quotedAnswerTool];

In [21]:

import { Document } from "@langchain/core/documents";

const outputParser2 = new JsonOutputKeyToolsParser({ keyName: "quoted_answer", returnSingle: true });
const llmWithTool2 = llm.bind({
  tools: tools2,
  tool_choice: quotedAnswerTool,
});
const answerChain2 = prompt.pipe(llmWithTool2).pipe(outputParser2);
const map2 = RunnableMap.from({
  question: new RunnablePassthrough(),
  docs: retriever,
})
// complete chain that calls the retriever -> formats docs to string -> runs answer subchain -> returns just the answer and retrieved docs.
const chain2 = map2
  .assign({ context: (input: { docs: Array<Document> }) => formatDocsWithId(input.docs) })
  .assign({ quoted_answer: answerChain2 })
  .pick(["quoted_answer", "docs"]);

In [22]:
await chain2.invoke("How fast are cheetahs?")

{
  quoted_answer: {
    answer: "Cheetahs can reach speeds of up to 70 mph.",
    citations: [
      {
        sourceId: 0,
        quote: "We’ve mentioned that these guys can reach speeds of up to 70 mph"
      },
      {
        sourceId: 2,
        quote: "The maximum speed cheetahs have been measured at is 114 km (71 miles) per hour, and they routinely r"... 72 more characters
      },
      {
        sourceId: 5,
        quote: "Cheetahs—the fastest land mammals on the planet—are able to reach speeds of up to 70 mph"
      }
    ]
  },
  docs: [
    Document {
      pageContent: "They are surprisingly graceful\n" +
        "Cheetahs are very lithe-they move quickly and full-grown adults weigh"... 824 more characters,
      metadata: {
        title: "How Fast Are Cheetahs - Proud Animal",
        source: "https://www.proudanimal.com/2024/01/27/fast-cheetahs/",
        score: 0.97272,
        images: null
      }
    },
    Document {
      pageContent: "The Science of Speed\n" +


LangSmith trace [here](https://smith.langchain.com/public/2a032bc5-5b04-4dc3-8d85-49e5ec7e0157/r)

## Direct prompting

Most models don't yet support function-calling. We can achieve similar results with direct prompting. Let's see what this looks like using an Anthropic chat model that is particularly proficient in working with XML:

### Setup

Install the LangChain Anthropic integration package:

```bash
npm install @langchain/anthropic
```

Add your Anthropic API key to your environment:

```bash
export ANTHROPIC_API_KEY=YOUR_KEY
```

In [23]:
import { ChatAnthropic } from "@langchain/anthropic";
import { ChatPromptTemplate } from "@langchain/core/prompts";

const anthropic = new ChatAnthropic({
  model: "claude-instant-1.2",
});
const system = `You're a helpful AI assistant. Given a user question and some web article snippets,
answer the user question and provide citations. If none of the articles answer the question, just say you don't know.

Remember, you must return both an answer and citations. A citation consists of a VERBATIM quote that
justifies the answer and the ID of the quote article. Return a citation for every quote across all articles
that justify the answer. Use the following format for your final output:

<cited_answer>
    <answer></answer>
    <citations>
        <citation><source_id></source_id><quote></quote></citation>
        <citation><source_id></source_id><quote></quote></citation>
        ...
    </citations>
</cited_answer>

Here are the web articles:{context}`;

const anthropicPrompt = ChatPromptTemplate.fromMessages([
  ["system", system],
  ["human", "{question}"]
]);

In [25]:
import { XMLOutputParser } from "@langchain/core/output_parsers";
import { Document } from "@langchain/core/documents";
import { RunnableLambda, RunnablePassthrough, RunnableMap } from "@langchain/core/runnables";

const formatDocsToXML = (docs: Array<Document>): string => {
  const formatted: Array<string> = [];
  docs.forEach((doc, idx) => {
    const docStr = `<source id="${idx}">
  <title>${doc.metadata.title}</title>
  <article_snippet>${doc.pageContent}</article_snippet>
</source>`
    formatted.push(docStr);
  });
  return `\n\n<sources>${formatted.join("\n")}</sources>`;
}

const format3 = new RunnableLambda({
  func: (input: { docs: Array<Document> }) => formatDocsToXML(input.docs)
})
const answerChain = anthropicPrompt
  .pipe(anthropic)
  .pipe(new XMLOutputParser())
  .pipe(
    new RunnableLambda({ func: (input: { cited_answer: any }) => input.cited_answer })
  );
const map3 = RunnableMap.from({
  question: new RunnablePassthrough(),
  docs: retriever,
});
const chain3 = map3.assign({ context: format3 }).assign({ cited_answer: answerChain }).pick(["cited_answer", "docs"])

In [26]:
await chain3.invoke("How fast are cheetahs?")

{
  cited_answer: [
    {
      answer: "Cheetahs can reach top speeds of between 60 to 70 mph."
    },
    {
      citations: [
        { citation: [Array] },
        { citation: [Array] },
        { citation: [Array] }
      ]
    }
  ],
  docs: [
    Document {
      pageContent: "A cheetah's muscular tail helps control their steering and keep their balance when running very fast"... 210 more characters,
      metadata: {
        title: "75 Amazing Cheetah Facts Your Kids Will Love (2024)",
        source: "https://www.mkewithkids.com/post/cheetah-facts-for-kids/",
        score: 0.97081,
        images: null
      }
    },
    Document {
      pageContent: "The maximum speed cheetahs have been measured at is 114 km (71 miles) per hour, and they routinely r"... 1048 more characters,
      metadata: {
        title: "Cheetah | Description, Speed, Habitat, Diet, Cubs, & Facts",
        source: "https://www.britannica.com/animal/cheetah-mammal",
        score: 0.96824,
        images: 

LangSmith trace [here](https://smith.langchain.com/public/bebd86f5-ae9c-49ea-bc26-69c4fdf195b1/r)

## Retrieval post-processing

Another approach is to post-process our retrieved documents to compress the content, so that the source content is already minimal enough that we don't need the model to cite specific sources or spans. For example, we could break up each document into a sentence or two, embed those and keep only the most relevant ones. LangChain has some built-in components for this. Here we'll use a [RecursiveCharacterTextSplitter](https://js.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter), which creates chunks of a specified size by splitting on separator substrings, and an [EmbeddingsFilter](https://js.langchain.com/docs/modules/data_connection/retrievers/contextual_compression#embeddingsfilter), which keeps only the texts with the most relevant embeddings.

In [27]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";
import { EmbeddingsFilter } from "langchain/retrievers/document_compressors/embeddings_filter";
import { OpenAIEmbeddings } from "@langchain/openai";
import { DocumentInterface } from "@langchain/core/documents";
import { RunnableMap, RunnablePassthrough } from "@langchain/core/runnables";

const splitter = new RecursiveCharacterTextSplitter({
  chunkSize: 400,
  chunkOverlap: 0,
  separators: ["\n\n", "\n", ".", " "],
  keepSeparator: false,
});

const compressor = new EmbeddingsFilter({
  embeddings: new OpenAIEmbeddings(),
  k: 10,
});

const splitAndFilter = async (input): Promise<Array<DocumentInterface>> => {
  const { docs, question } = input;
  const splitDocs = await splitter.splitDocuments(docs);
  const statefulDocs = await compressor.compressDocuments(splitDocs, question);
  return statefulDocs;
};

const retrieveMap = RunnableMap.from({
  question: new RunnablePassthrough(),
  docs: retriever,
});

const retrieve = retrieveMap.pipe(splitAndFilter);
const docs = await retrieve.invoke("How fast are cheetahs?");
for (const doc of docs) {
  console.log(doc.pageContent, "\n\n");
}

The maximum speed cheetahs have been measured at is 114 km (71 miles) per hour, and they routinely reach velocities of 80–100 km (50–62 miles) per hour while pursuing prey.
cheetah,
(Acinonyx jubatus), 


The science of cheetah speed
The cheetah (Acinonyx jubatus) is the fastest land animal on Earth, capable of reaching speeds as high as 75 mph or 120 km/h. Cheetahs are predators that sneak up on their prey and sprint a short distance to chase and attack.
 Key Takeaways: How Fast Can a Cheetah Run?
Fastest Cheetah on Earth 


Built for speed, the cheetah can accelerate from zero to 45 in just 2.5 seconds and reach top speeds of 60 to 70 mph, making it the fastest land mammal! Fun Facts
Conservation Status
Cheetah News
Taxonomic Information
Animal News
NZCBI staff in Front Royal, Virginia, are mourning the loss of Walnut, a white-naped crane who became an internet sensation for choosing one of her keepers as her mate. 


Scientists calculate a cheetah's top speed is 75 mph, but the fast

LangSmith trace [here](https://smith.langchain.com/public/1bb61806-7d09-463d-909a-a7da410e79d4/r)

In [28]:
const chain4 = retrieveMap
  .assign({ context: formatDocs })
  .assign({ answer: answerChain })
  .pick(["answer", "docs"]);

In [29]:
// Note the documents have an article "summary" in the metadata that is now much longer than the
// actual document page content. This summary isn't actually passed to the model.
await chain4.invoke("How fast are cheetahs?")

{
  answer: [
    {
      answer: "\n" +
        "Cheetahs are the fastest land animals. They can reach top speeds of around 75 mph (120 km/h) and ro"... 74 more characters
    },
    { citations: [ { citation: [Array] }, { citation: [Array] } ] }
  ],
  docs: [
    Document {
      pageContent: "The maximum speed cheetahs have been measured at is 114 km (71 miles) per hour, and they routinely r"... 1048 more characters,
      metadata: {
        title: "cheetah - Encyclopedia Britannica | Britannica",
        source: "https://www.britannica.com/animal/cheetah-mammal",
        score: 0.97059,
        images: null
      }
    },
    Document {
      pageContent: "Contact Us − +\n" +
        "Address\n" +
        "Smithsonian's National Zoo & Conservation Biology Institute  3001 Connecticut"... 1343 more characters,
      metadata: {
        title: "Cheetah",
        source: "https://nationalzoo.si.edu/animals/cheetah",
        score: 0.95102,
        images: null
      }
    },
    Docu

LangSmith trace [here](https://smith.langchain.com/public/f93302e6-a31b-454e-9fc7-94fb4a931a9d/r)

## Generation post-processing

Another approach is to post-process our model generation. In this example we'll first generate just an answer, and then we'll ask the model to annotate it's own answer with citations. The downside of this approach is of course that it is slower and more expensive, because two model calls need to be made.

Let's apply this to our initial chain.

In [6]:
import { StructuredTool } from "@langchain/core/tools";
import { formatToOpenAITool } from "@langchain/openai";
import { z } from "zod";

class AnnotatedAnswer extends StructuredTool {
  name = "annotated_answer";

  description = "Annotate the answer to the user question with quote citations that justify the answer";

  schema = z.object({
    citations: z.array(citationSchema).describe("Citations from the given sources that justify the answer."),
  })

  _call(input: z.infer<typeof this["schema"]>): Promise<string> {
    return Promise.resolve(JSON.stringify(input, null, 2));
  }
}

const annotatedAnswerTool = formatToOpenAITool(new AnnotatedAnswer());

const llmWithTools5 = llm.bind({
  tools: [annotatedAnswerTool],
  tool_choice: annotatedAnswerTool,
})

In [27]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { RunnableSequence } from "@langchain/core/runnables";
import { JsonOutputKeyToolsParser } from "langchain/output_parsers";
import { RunnableMap, RunnablePassthrough } from "@langchain/core/runnables";
import { AIMessage, ToolMessage } from "@langchain/core/messages";

const prompt5 = ChatPromptTemplate.fromMessages([
  ["system", "You're a helpful AI assistant. Given a user question and some web article snippets, answer the user question. If none of the articles answer the question, just say you don't know.\n\nHere are the web articles:{context}"],
  ["human", "{question}"],
  new MessagesPlaceholder({
    variableName: "chat_history",
    optional: true,
  }),
  new MessagesPlaceholder({
    variableName: "toolMessage",
    optional: true,
  })
]);

const answerChain5 = prompt5.pipe(llmWithTools5);
const annotationChain = RunnableSequence.from([
  prompt5,
  llmWithTools5,
  new JsonOutputKeyToolsParser({ keyName: "annotated_answer", returnSingle: true }),
  (input: any) => input.citations
]);
const map5 = RunnableMap.from({
  question: new RunnablePassthrough(),
  docs: retriever,
});
const chain5 = map5
  .assign({ context: formatDocs })
  .assign({ aiMessage: answerChain5 })
  .assign({
    chat_history: (input) => input.aiMessage,
    toolMessage: (input) => new ToolMessage({
      tool_call_id: input.aiMessage.additional_kwargs.tool_calls[0].id,
      content: input.aiMessage.additional_kwargs.content ?? "",
    })
  })
  .assign({
    annotations: annotationChain,
  })
  .pick(["answer", "docs", "annotations"]);

In [28]:
await chain5.invoke("How fast are cheetahs?")

{
  docs: [
    Document {
      pageContent: "They are surprisingly graceful\n" +
        "Cheetahs are very lithe-they move quickly and full-grown adults weigh"... 824 more characters,
      metadata: {
        title: "How Fast Are Cheetahs - Proud Animal",
        source: "https://www.proudanimal.com/2024/01/27/fast-cheetahs/",
        score: 0.96021,
        images: null
      }
    },
    Document {
      pageContent: "Contact Us − +\n" +
        "Address\n" +
        "Smithsonian's National Zoo & Conservation Biology Institute  3001 Connecticut"... 1343 more characters,
      metadata: {
        title: "Cheetah | Smithsonian's National Zoo and Conservation Biology Institute",
        source: "https://nationalzoo.si.edu/animals/cheetah",
        score: 0.94798,
        images: null
      }
    },
    Document {
      pageContent: "The science of cheetah speed\n" +
        "The cheetah (Acinonyx jubatus) is the fastest land animal on Earth, cap"... 738 more characters,
      metada

LangSmith trace [here](https://smith.langchain.com/public/f4ca647d-b43d-49ba-8df5-65a9761f712e/r)